In [25]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from analysis import WordList
from pyspark.sql.types import IntegerType, DateType
from pyspark import StorageLevel

import pyspark.sql.functions as F
from operator import add

In [3]:
spark = SparkSession \
        .builder \
        .master('spark://10.10.28.172:7077') \
        .appName('experimenting with runtimes') \
        .enableHiveSupport() \
        .config("spark.pyfiles", "analysis.py") \
        .getOrCreate()

# Preventing errors of having unused settings in hive-site.xml
spark.sparkContext.setLogLevel('OFF')
# path of dependency file(s)
spark.sparkContext.addPyFile("/home/ubuntu/twitter_sentiment/code/analysis/analysis.py")

spark.sql('show databases').show()

22/04/27 20:27:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+------------+
|   namespace|
+------------+
|     default|
|twitter_data|
+------------+



In [4]:
spark.sql('use twitter_data')
spark.sql('show tables').show()

+------------+--------------------+-----------+
|    database|           tableName|isTemporary|
+------------+--------------------+-----------+
|twitter_data|  daily_sentiment_mr|      false|
|twitter_data|      processed_data|      false|
|twitter_data|            raw_data|      false|
|twitter_data|             results|      false|
|twitter_data|    textblob_results|      false|
|twitter_data|       vader_results|      false|
|twitter_data|wordlist2477_results|      false|
+------------+--------------------+-----------+



In [5]:
wl = WordList(r'2477')
wordlist = wl.dict  # If you want to use the dictionary directly
print('test score: ' + str(wl.analyze('hello this is a stupid test')))

test score: -2


In [6]:
# Implementation with MR
# d = spark.sql('select to_date(created_at) as date, split(text, " ") as word from processed_data')
# rdd = d.rdd
# rdd.persist(StorageLevel.MEMORY_AND_DISK)

# rdd.flatMapValues(lambda x: x) \
#     .mapValues(lambda x: wordlist[x]) \
#     .reduceByKey(add) \
#     .toDF(['date', 'total_sentiment']) \
#     .show()

In [7]:
udf_wl_analyze = udf(lambda text: wl.analyze(text), IntegerType())
df = spark.sql('select created_at as date, text from processed_data')
df.persist(StorageLevel.MEMORY_AND_DISK)

DataFrame[date: timestamp, text: string]

In [37]:
df.withColumn('sentiment', udf_wl_analyze('text')). \
    groupBy(F.window('date', '1 day')). \
    sum(). \
    select([
        F.to_date(F.col('window.start')).alias('date'), 
        F.col('sum(sentiment)').alias('total_sentiment')]). \
    write.mode('overwrite').saveAsTable('testing')
    # select('d')